# Model 05 inference


## Setup

### Working directory

In [1]:
# Change the working directory to project root
from pathlib import Path
import os
ROOT_DIR = Path.cwd()
while not ROOT_DIR.joinpath("src").exists():
    ROOT_DIR = ROOT_DIR.parent
os.chdir(ROOT_DIR)

### Dependencies

In [2]:
# Imports and dependencies
import torch
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss
from torch.optim import AdamW
from torch.optim.lr_scheduler import LinearLR
from torcheval.metrics import BinaryAccuracy, BinaryF1Score

from src.torch_utils import get_torch_device
import json
from dataclasses import dataclass
from typing import List, Union, Tuple, Dict
from tqdm import tqdm
import random
import numpy as np
from datetime import datetime
from sklearn.model_selection import ParameterGrid

from src.model_05 import BertCrossEncoderClassifier
from src.data import RetrievalWithShortlistDataset, RetrievalDevEvalDataset, \
    InferenceClaims
from src.logger import SimpleLogger

TORCH_DEVICE = get_torch_device()

/opt/homebrew/Caskroom/miniconda/base/envs/comp90042_project/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Torch device is 'mps'


### File paths

In [3]:
MODEL_PATH = ROOT_DIR.joinpath("./result/models/*")
DATA_PATH = ROOT_DIR.joinpath("./data/*")
LOG_PATH = ROOT_DIR.joinpath("./result/logs/*")
SHORTLIST_PATH = ROOT_DIR.joinpath("./result/pipeline/shortlisting_v2/*")
OUTPUT_PATH = ROOT_DIR.joinpath("./result/pipeline/retrieval_classif/*")

run_time = datetime.now().strftime('%Y_%m_%d_%H_%M')

## Load model

Use a blank pre-trained

In [4]:
# model = BertCrossEncoderClassifier(
#     pretrained_name="bert-base-uncased",
#     n_classes=2,
#     device=TORCH_DEVICE
# )

Or load fine-tuned

In [6]:
MODEL_SAVE_PATH = MODEL_PATH.with_name("model_05_bert_cross_encoder_retrieval_2023_05_08_17_06.pth")
with open(MODEL_SAVE_PATH, mode="rb") as f:
    model = torch.load(f, map_location=TORCH_DEVICE)

## Inference run code

In [7]:
@dataclass
class EvidenceScore:
    evidence_id:str
    score:float
    
    def to_list(self) -> List[str]:
        return [self.evidence_id, str(self.score)]
    
    def to_dict(self) -> Dict[str, str]:
        return {"evidence_id": self.evidence_id, "score": str(self.score)}

In [18]:
def inference_run(
    claims_path:Path,
    evidence_path:Path,
    claims_shortlist_path:Path = None,
    save_path:Path = None,
    batch_size:int = 64
):
    # Generate claims iterations
    inference_claims = InferenceClaims(claims_path=claims_path)
    
    # Cumulator
    claim_predictions = dict()
    
    # Load from save to continue inference if exists
    saved_predictions = dict()
    if save_path.exists():
        with open(save_path, mode="r") as f:
            saved_predictions.update(json.load(fp=f))
    
    for claim_id, claim_text in inference_claims:
        
        # Skip if a save entry exists
        if claim_id in saved_predictions.keys():
            print(f"skipping {claim_id}, done previously")
            continue
        
        # Create a save entry
        claim_predictions[claim_id] = {
            "claim_text": claim_text,
            "evidences": []
        }
        
        # Generate claims-evidence inference interations
        infer_data = RetrievalWithShortlistDataset(
            claim_id=claim_id,
            claims_paths=[claims_path],
            claims_shortlist_paths=[claims_shortlist_path],
            evidence_path=evidence_path,
            pos_label=0, # Wont' matter what label we use for inference
            neg_label=0,
            n_neg_samples=9999999999,
            shuffle=False,
            seed=42,
            verbose=True
        )
        infer_dataloader = DataLoader(
            dataset=infer_data,
            shuffle=False,
            batch_size=batch_size
        )
        
        print(f"running inference for {claim_id} n={len(infer_data)}")
    
        # Set model mode to evaluation
        model.eval()
        
        infer_batches = tqdm(infer_dataloader, desc="infer batches")
        for batch in infer_batches:
            claim_texts, evidence_texts, labels, batch_claim_ids, batch_evidence_ids = batch
            texts = list(zip(claim_texts, evidence_texts))
            
            # Forward
            output, logits, seq = model(
                texts=texts,
                normalize_text=True,
                max_length=512,
                dropout=None
            )
            
            # Get score as probabilities for label 1
            batch_scores = output[:, 1]
            
            for c_id, e_id, score in zip(
                batch_claim_ids, batch_evidence_ids,
                batch_scores.cpu().detach().numpy()
            ):
                claim_predictions[c_id]["evidences"].append(EvidenceScore(
                    evidence_id=e_id,
                    score=score
                ))
            
            continue
    
        # Save on every claim_id
        if save_path:
            # Retrieve at most 5 top predicted evidences by score
            claim_predictions_output = dict()
            for claim_id_, claim_ in claim_predictions.items():
                claim_ = claim_.copy()
                claim_["evidences"] = [
                    # (evidence_score.evidence_id, str(round(evidence_score.score, 3)))
                    evidence_score.evidence_id
                    for evidence_score in sorted(
                        claim_["evidences"],
                        key=lambda es: es.score,
                        reverse=True
                    )
                ][:5]
                claim_predictions_output[claim_id_] = claim_
            
            # Make a copy of existing saved results and add the new results
            # for this run
            save_dict = saved_predictions.copy()
            save_dict.update(claim_predictions_output)

            with open(save_path, mode="w") as f:
                json.dump(obj=save_dict, fp=f)
                print(f"saved to: {save_path}")
    
        continue
    return

## Dev inference

In [20]:
inference_run(
    claims_path=DATA_PATH.with_name("dev-claims.json"),
    evidence_path=DATA_PATH.with_name("evidence.json"),
    claims_shortlist_path=SHORTLIST_PATH.with_name("dev_retrieved_evidences_max_1000_rel.json"),
    # evidence_shortlist=NER_PATH.with_name("shortlist_dev_claim_evidence_retrieved.json"),
    save_path=OUTPUT_PATH.with_name("model_05_bert_base_2023_05_08_17_06_dev_shortlist_max_1000_with_rel.json"),
    batch_size=24
)

loaded inference claims n=154
Torch device is 'mps'


claims: 100%|██████████| 154/154 [00:00<00:00, 115881.38it/s]


generated dataset n=873
running inference for claim-752 n=873


infer batches: 100%|██████████| 37/37 [00:14<00:00,  2.51it/s]


saved to: /Users/johnsonzhou/git/comp90042-project/result/pipeline/retrieval_classif/model_05_bert_base_2023_05_08_17_06_dev_shortlist_max_1000_with_rel.json
Torch device is 'mps'


...

saved to: /Users/johnsonzhou/git/comp90042-project/result/pipeline/retrieval_classif/model_05_bert_base_2023_05_08_17_06_dev_shortlist_max_1000_with_rel.json
Torch device is 'mps'


claims: 100%|██████████| 154/154 [00:00<00:00, 124985.07it/s]


generated dataset n=855
running inference for claim-1021 n=855


infer batches: 100%|██████████| 36/36 [00:14<00:00,  2.55it/s]

saved to: /Users/johnsonzhou/git/comp90042-project/result/pipeline/retrieval_classif/model_05_bert_base_2023_05_08_17_06_dev_shortlist_max_1000_with_rel.json
